In [1]:
!nvidia-smi

Sat Nov 23 12:11:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.51.01              Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              1W /   60W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
SCRIPT_DIR = os.path.abspath("")

In [2]:
from dataset_util import merge

train_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/train'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/train'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/train'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/train')]
taco_index = [0,1]

merge.merge_yolo(train_sets, os.path.join(SCRIPT_DIR, 'merged_data/train'), taco_index)

output_dir = os.path.join(SCRIPT_DIR, 'merged_data')
yaml_content = """train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Person', 'Trash']
"""

with open(f"{output_dir}/data.yaml", "w") as yaml_file:
    yaml_file.write(yaml_content)

Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO/data/train completed, Counter: 0
Dataset /mnt/c/Adrianov/Projects/trash-detector/TACO_TN_UAV/train completed, Counter: 1
Dataset /mnt/c/Adrianov/Projects/trash-detector/coco_human/data/train completed, Counter: 2
Dataset /mnt/c/Adrianov/Projects/trash-detector/people_detection/data/train completed, Counter: 3
Datasets merged into: /mnt/c/Adrianov/Projects/trash-detector/merged_data/train


In [ ]:
validation_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/validation'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/validation'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/validation'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/validation')]
taco_index = [0,1]

merge.merge_yolo(validation_sets, os.path.join(SCRIPT_DIR, 'merged_data/validation'), taco_index)


In [ ]:
test_sets = [os.path.join(SCRIPT_DIR, 'TACO/data/test'), 
              os.path.join(SCRIPT_DIR, 'TACO_TN_UAV/test'), 
              os.path.join(SCRIPT_DIR, 'coco_human/data/test'),
              os.path.join(SCRIPT_DIR, 'people_detection/data/test')]
taco_index = [0,1]

merge.merge_yolo(validation_sets, os.path.join(SCRIPT_DIR, 'merged_data/test'), taco_index)


In [5]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt") 
merged_data = os.path.join(SCRIPT_DIR, 'merged_data/data.yaml')
output_dir = os.path.join(SCRIPT_DIR, 'model')
results = model.train(name='Trash Detector', data=merged_data, epochs=100, 
                      imgsz=640, batch=16, device=0, project=output_dir, 
                      resume=False, plots=True, exist_ok=True,
                      save_period=10)

New https://pypi.org/project/ultralytics/8.3.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.29 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/mnt/c/Adrianov/Projects/trash-detector/TACO/data/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=/mnt/c/Adrianov/Projects/trash-detector/model, name=TACO, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=Fal

train: Scanning /mnt/c/Adrianov/Projects/trash-detector/TACO/data/train/labels.cache... 5369 images, 0 backgrounds, 0 corrupt: 100%|██████████| 
val: Scanning /mnt/c/Adrianov/Projects/trash-detector/TACO/data/valid/labels.cache... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801


Plotting labels to /mnt/c/Adrianov/Projects/trash-detector/model/TACO/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000455, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /mnt/c/Adrianov/Projects/trash-detector/model/TACO
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.97G      1.363      4.209      1.239         39        640: 100%|██████████| 336/336 [01:17<00:00,  4.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:11<00:00,  2.35it/s]


                   all        801       2318      0.186      0.151     0.0577     0.0402

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        801       2318      0.578      0.409      0.442      0.335

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     22/100      2.71G       1.11      1.375      1.086         38        640: 100%|██████████| 336/336 [01:25<00:00,  3.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:09<00:00,  2.63it/s]


                   all        801       2318      0.652      0.405      0.457      0.343

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     23/100      2.89G      1.114      1.345      1.092        101        640:  49%|████▉     | 166/336 [00:36<00:37,  4.51it/s]


KeyboardInterrupt: 

In [6]:
2 in [0,1]

False